```{image} https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png 
:width: 140px
:align: left
```
## Image Processing Laboratory Notebooks
---

This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-511 Image Processing I**](https://moodle.epfl.ch/course/view.php?id=522) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2023.

**Authors**: 
    [Pol del Aguila Pla](mailto:pol.delaguilapla@epfl.ch), 
    [Kay Lächler](mailto:kay.lachler@epfl.ch),
    [Alejandro Noguerón Arámburu](mailto:alejandro.nogueronaramburu@epfl.ch),
    [Yan Liu](mailto:yan.liu@epfl.ch), and
    [Daniel Sage](mailto:daniel.sage@epfl.ch).
    
---

# Lab 2.2: Filtering Applications
**Released**: Thursday November 9, 2023

**Submission**: Monday November 20, 2023 before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=522)

**Grade weight**: Lab 2 (16 points), 10% of the overall grade

**Related lectures**: Chapter 3

Double-click on this cell, fill your name and SCIPER number below to verify your identity in Noto and set the seed for random results.
:::{attention} Please write down your name and SCIPER! 
### Student Name: 
### SCIPER: 
:::

In [ ]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

### <a name="imports_"></a> Imports
In the next cells we import the libraries and load the images that we will use throughout the lab.

In [ ]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io 
import ipywidgets as widgets
import skimage
from skimage import filters
from interactive_kit import imviewer as viewer

# Load images to be used in this exercise 
bikesgray = io.imread('images/bikesgray.tif').astype('float64')
camera = io.imread('images/camera-16bits.tif').astype('float64')
spots = io.imread('images/spots.tif').astype('float64')
hela = io.imread('images/hela-DIC.tif').astype('float64')

In [ ]:
%use javascript
%get bikesgray camera spots
// import IPLabImageAccess as Image
var Image = require('image-access')

# Filtering applications (7 points)

After the first part of the lab, we expect you to feel comfortable with the basics of filtering. In this part we will look in detail at the implementation of a Gaussian filter, as well as some of its direct applications. Gaussian filters are known to be near-optimal smoothing filters, and represent perhaps the most used preprocessing step in image processing to improve robustness in a workflow and to denoise images.

(ToC_2_FilteringApplications)=
## Table of contents
1. [Gaussian filter](#gaussian-filter)
    1. [Implementation of a 2D Gaussian filter](#implementation-of-a-gaussian-filter) (**4 points**)
    2. [Gaussian filter in Python](#gaussian-filter-in-python)
2. [Application: Segmentation of a DIC](#application-segmentation-of-a-dic)
    1. [Implementation](#implementation) (**3 points**)
    2. [Understanding the effect of the parameters](#understanding-the-effect-of-the-parameters)

### Visualize images
Get familiar with the images that you are going to use by running the next cell.

In [ ]:
%use sos
# Declare image_list for ImageViewer
image_list = [bikesgray, camera, spots]

imgs_viewer = viewer(image_list, widgets=True, hist=True)

(gaussian-filter)=
# 1. Gaussian filter (4 points)
[Back to table of contents](#ToC_2_FilteringApplications)

An [isotropic 2D Gaussian](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) is

$$h_\sigma(x,y) = \frac{1}{2\pi\sigma^2}\exp\left(-\frac{x^2 + y^2}{2\sigma^2}\right) \overset{\mathrm{separability}}{=} \frac{1}{\sqrt{2\pi}\sigma}\exp\left(-\frac{x^2}{2\sigma^2}\right) \frac{1}{\sqrt{2\pi}\sigma}\exp\left(-\frac{ y^2}{2\sigma^2}\right) \,,$$

where $\sigma$ is the standard deviation and controls the smoothing strength.

In this section you will implement a 2D Gaussian filter with impulse response $h_{\sigma}[m,n]$, which discretizes $h_\sigma(x,y)$ between $[-\lceil3\sigma\rceil,\lceil 3\sigma\rceil]$ in $x$ and $y$. Here, $\lceil x \rceil$ refers to the smallest integer larger than a given $x\in\mathbb{R}$. Additionally, choose the size of the filter to be $N = 2\lceil 3\sigma \rceil+1$ (hence, $N$ is always odd), and ensure the impulse response adds up to $1$, using appropriate normalization.

(implementation-of-a-gaussian-filter)=
## 1.A. Implementation of a 2D Gaussian filter (4 points)
[Back to table of contents](#ToC_2_FilteringApplications)

For **3 points**, implement the function `gaussian(img, sigma)` that convolves an image with a Gaussian filter using a **separable implementation** in JavaScript. Take advantage of the `filter1D` function from lab 2.1 that we have copied here for you in the next cell.

:::{hint}
* You can use [Math library](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Math) to access different mathematical functions.
* The first argument to the `Image` constructor is the **height** and the second is the **width**: `new Image(height, width)` or `new Image([height, width])`. Feel free to go back to Lab 0 to review the basic usage of the ImageAccess class.
:::

In [ ]:
%use javascript
// function that performs a gaussian filter with sigma on img
function gaussian(img, sigma){
    // declare output variable
    var output = new Image(img.shape());
    
    // Define normalized mask
    // YOUR CODE HERE
    
    // Filter using separable implementation (hint: your mask should be 1D)
    // You can use the filter1D function defined below
    // YOUR CODE HERE
    return output
}

// function that applies a 1D filter
function filter1D(img, mask){
    // transpose the input variables if necessary
    if(img.nx == 1){
        img.transposeImage();
    }
    if(mask.nx == 1){
        mask.transposeImage();
    }
    // create the output image
    var output = new Image(img.shape());
    // iterate through all pixels
    for(var x = 0; x < img.nx; x++){
        // get the neighbourhood around position x
        var neigh = img.getNbh(x, 0, mask.nx, 1);
        // declare a variable to store the values of the convolution. 
        var val = 0;
        // iterate through the neighbourhood
        for(var i = 0; i < neigh.nx; i++){
            // perform convolution
            val += neigh.getPixel(i, 0) * mask.getPixel(mask.nx - 1 - i, 0);
        }
        // set value in output array
        output.setPixel(x, 0, val);
    }
    return output
}

We have designed a quick test for you to evaluate your method, applying it to a $3 \times 3$ impulse image. Run the following cell and check that your output has all the desired properties of a Gaussian.

In [ ]:
%use javascript
// define the impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);

// apply filter to previously defined impulse
var impulse_gaussian = gaussian(impulse, 0.5);

// look at result, verify that it has the properties of a Gaussian
console.log('Your impulse Gaussian:\n' + impulse_gaussian.visualize());

// this assertion checks proper behaviour: that the center is the maximum, and that two pixels in equivalent positions have the same values.
if(impulse_gaussian.getPixel(1, 1) < impulse_gaussian.getPixel(1, 2) || impulse_gaussian.getPixel(1, 0) !== impulse_gaussian.getPixel(1, 2)){
    console.log('WARNING!!!\nThere are still some mistakes with your implementation! Look at the sanity checks to understand the mistakes');
}else{
    console.log('The symmetry of the Gaussian seems good.');
}

// check normalization
var sum = 0
for(var x = 0; x < impulse_gaussian.nx; x++){
    for (var y = 0; y < impulse_gaussian.ny; y++){
        sum += impulse_gaussian.getPixel(x, y);
    }
}
if(Math.abs(sum - 1) > 1e-5){
    console.log("WARNING!!\nNormalization not correct");
}else{
    console.log("Well done! The output sums up to approximately 1.");
}

Now that you have tested your Gaussian filter, you can apply it to the image `bikesgray`. Use different values of $\sigma$ (you can change it in the next cell). Look at the evolution of the mean and the standard deviation (you can get them from the statistics box in the `viewer`, or you can use the functions `np.mean` and `np.std`). Then, answer the two multiple choice questions.

Run and modify the two following cells to apply Gaussian filters with different $\sigma$ values to `bikesgray` and view the result. 

:::{important} 
Don't forget to change $\sigma$ to a normal (small) value after your exploration!
:::

In [ ]:
%use javascript
%put bikesgray_gaussian1 bikesgray_gaussian5

// apply filter to Image object. To try different sigma values, change the variables or declare more. 
var bikesgray_gaussian1 = gaussian(new Image(bikesgray), 1).toArray()
var bikesgray_gaussian5 = gaussian(new Image(bikesgray), 5).toArray()

In [ ]:
%use sos

# Declare parameters for ImageViewer. If you want to visualize more sigma values, update the previous cell and these lists accordingly
image_list_blur = [bikesgray, bikesgray_gaussian1, bikesgray_gaussian5]
title_list_blur = ['Original', 'Sigma: 1', 'Sigma: 5']
# Make sure that the object used is a numpy array
for i in range(len(image_list_blur)):
    image_list_blur[i] = np.array(image_list_blur[i])

# To allow a direct comparison of the images.
plt.close('all')
blurred_bikesgray_viewer = viewer(image_list_blur, title=title_list_blur, hist=True)

### Multiple Choice Question

After modifying the two cells above and visualizing the results, answer the next two questions (worth **0.5 points** each).

* Q1: How would you expect the Fourier transform of an image to change after applying a Gaussian filter?
    1. It will show lower values for higher frequencies.
    2. It will show higher values for higher frequencies.
    3. It will show lower values for lower frequencies.
    4. It will not change.


* Q2: What will be the output image when $\sigma\rightarrow \infty$? What type of filter would that be?
    1. An image equal to the original. It would be an all-pass filter.
    2. A constant image. It would be a high-pass filter.
    3. A 2D Gaussian. It would be a band-pass filter.
    4. A constant image. It would be a low-pass filter.

Modify the variables `answer_one` and `answer_two` in the next cell to match your choices. The second and third cells are for you to make sure that your answer is in the valid range (they should not raise any error).

In [ ]:
%use sos
# Modify these variables
answer_one = None
answer_two = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity test
if not answer_one in [1, 2, 3, 4]:
    print('WARNING!\nAnswer one of 1, 2, 3 or 4.')

In [ ]:
%use sos
# Sanity test
if not answer_two in [1, 2, 3, 4]:
    print('WARNING!\nAnswer one of 1, 2, 3 or 4.')

(gaussian-filter-in-python)=
## 1.B. Gaussian filter in Python
[Back to table of contents](#ToC_2_FilteringApplications)

There are several implementations of Gaussian filters in Python. In this section, we will use the `scikit-image` implementation [`skimage.filters.gaussian()`](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.gaussian). This function is a wrapper around the SciPy ndimage function [`scipy.ndi.gaussian_filter()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html), feel free to check both documentations to inform yourself.

Here is an example of how to use it:

```python
output = skimage.filters.gaussian(input, sigma=10, mode='reflect', truncate=3, preserve_range=True)
```
The above line specifies that we apply on the `input` image a Gaussian filter with $\sigma=10$, reflective boundary condition, truncated to $3\sigma$ range and keep the original range of values of `input`.


In the next cells, we will apply it to the image `bikesgray` and compare it to your implementation. Run the next cell to get a blurred version of bikesgray.

In [ ]:
%use sos
# Apply Gaussian filtering in python
bikesgray_gaussian_skimage = skimage.filters.gaussian(bikesgray, sigma=10 , mode='reflect', truncate=3, preserve_range=True)
gaussian_viewer = viewer(bikesgray_gaussian_skimage)

Now, we will compare it to your implementation in JavaScript to make sure that they are equivalent (up to errors on the order of $10^{-14}$). For this, we call the `gaussian` function you implemented with the image bikesgray, also for $\sigma = 10$. 

Run the next cell to get the variable `bikesgray_gaussian10`, and the one below it to make the comparison in Python. Note that, if the images are not the same, an `ImageViewer` will pop up, showing you in red the regions that differ the most. If necessary, you can use this information to try to find your mistakes. 

In [ ]:
%use javascript
%put bikesgray_gaussian10
// apply filter to Image object
var bikesgray_gaussian10 = gaussian(new Image(bikesgray), 10).toArray()

Now we will look at the results of the Python code and your code, and at their differences. Look at the range of values in the histogram to verify the scale of the differences.

In [ ]:
%use sos
# Make sure that the one imported from JavaScript is a numpy array
bikesgray_gaussian10 = np.array(bikesgray_gaussian10)

# Declare parameters of viewer
image_list = [bikesgray_gaussian10, bikesgray_gaussian_skimage, np.abs(bikesgray_gaussian_skimage - bikesgray_gaussian10)]
title_list = ['JS', 'Skimage', 'Difference']

# We call the viewer with clip_range = [0, 1] to compare the difference with respect to the originals
plt.close('all')
if not np.allclose(bikesgray_gaussian10, bikesgray_gaussian_skimage):
    print('The results of your Gaussian filter do not match Skimage results! Look at the red areas in the viewer to see where you might have gone wrong.')
    skimage_gaussian_viewer = viewer([bikesgray_gaussian10, bikesgray_gaussian_skimage], title=['JS', 'Skimage'], widgets=True, compare=True)
else :
    print('Seems like your Gaussian filter is correct!')

# 2. Application: Segmentation of a DIC (3 points)
[Back to table of contents](#ToC_2_FilteringApplications)

Now that you have implemented several filters and you master the concepts behind digital filtering, we are going to lead you in a real application of Gaussian filtering. In this application, you will work on an image of *DIC Microscopy*, and you will use Gaussian filters to estimate the *local standard deviation*.

<div class = 'alert alert-info'>
<b>About the technique</b>: <a href='https://en.wikipedia.org/wiki/Differential_interference_contrast_microscopy'>Differential interference contrast (DIC) microscopy</a> is a <i>label-free</i> microscopy technique used to visualize transparent samples. It gains informantion about the optical path length of the sample through interferometry, which allows it to <b>detect the shape of a transparent sample</b> in ways that traditional microscopy cannot. It produces <b>shadow-like and reflection-like structures. Uniform regions in the sample will produce uniform values, whereas regions with high variations will present variations in the values</b>. However, if we want to detect where in the image is our sample and separate it from the rest of the image, it is impossible to do so using simple thresholding. A classical solution is to <b>threshold the local standard deviation of the input image</b>. With proper values of threshold $t$ and of standard deviation $\sigma$, the sample will be completely extracted from the background.
</div> 

Take a look at the detailed workflow of *DIC Segementation* in the next figure.
<table><tr><td>
<p align="center" style="padding: 10px">
<img src="images/DIC_segmentation_workflow.jpg" width="600"><br>
<em style="color: grey">Segmentation of a DIC workflow: A Gaussian filter is applied to the original image <i><b>f</b></i>, to obtain the local mean <i><b>m</b></i>. Then the local $\sigma$ <i><b>s</b></i> is obtained by substracting <i><b>f-m</b></i>, and applying the square, a Gaussian and the square root operators. The final image is constructed by placing <i><b>f</b></i> in the red and green channels, and the thresholded <i><b>s</b></i> in the blue channel.</em></p> </td></tr></table>

## 2.A. Implementation (3 points)
[Back to table of contents](#ToC_2_FilteringApplications)

We propose to implement the algorithm sketched in the previous figure. It displays **in the blue channel the thresholded local standard deviation, and in the red and green channels the original image**. The two parameters $\sigma$ and $t$ are given as inputs of the workflow. For this you will write the functions `local_mean`, `local_std` and `segment_dic`. **Each of <i>m</i>, <i>s</i> and the final image will give you $1$ point for a total of $3$**. 

`local_mean(img, sigma)` takes as parameters: 
 * `img`: The original DIC Microscopy image, and 
 * `sigma`: The standard deviation to use during Gaussian filtering.

It returns the local mean (**<i>m</i> in the diagram**).

`local_std(img, mean, sigma)` takes as parameters:
 * `img`: The original DIC Microscopy image,
 * `mean`: The local mean of the original DIC Microscopy image, and 
 * `sigma`: The standard deviation to use during Gaussian filtering. 
 
It returns the local $\sigma$ (**<i>s</i> in the diagram**).

Finally, `segment_dic(img, std, t)` takes as parameters:
 * `img`: The original DIC Microscopy image,
 * `std`: The local $\sigma$ of the original DIC Microscopy image, and 
 * `t`: The threshold, in the range $[0, 255]$.
 
And returns the segmented image.
 
<div class = 'alert alert-success '>
<b>Hints</b>:<ul><li>Remember to keep the final image in the range $[0, 255]$.</li><li>To assign a grayscale $(m\times n)$ image <code>grayscale_img</code> to a channel <code>channel</code> of an RGB image you can simply do <code>RGB_img[:, :, channel] = graysale_img</code>.</li><li>Use the comments in the function <code>segment_dic</code> to guide your solution.</li></ul></div>

<div class="alert alert-warning">
  <b>Technical notes and requirements</b>:<ul><li>For RGB images, matplotlib requires the data to be in the range <i>[0, 1]</i> for images of data type <code>float</code>, and between <i>[0, 255]</i> for images of data type <code>int</code>. When working with RGB images, make sure to cast your output to one of these data types (e.g. <code>output = output.astype(int)</code>) and range. In the following cell we cast the datatype for you, but <b>you have to make sure that the range is the appropriate</b>.</li><li>in Section <a href="#1.B.-Gaussian-filter-in-Python">1.B.</a> we presented you the <code>scikit-image</code> implementation of the Gaussian filter, which is simple and clear ($2$ strong points of SciKit-Image). However, as you will see later, OpenCV and SciPy tend to be faster (e.g., SciKit-Image's implementation is actualy just a wrapper around SciPy's), and have different strong points. Of course we will accept any correct implementation, but we encourage you to find <a href='https://docs.opencv.org/4.x/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1'>OpenCV</a>'s and <a href='https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html'>SciPy</a>'s implementations (documentation in links) and use the one you prefer! <b>Make sure you use the correct parameters, like <code>preserve_range = True</code> option (or equivalent), truncating the filter at $3\sigma$ (so $N = 2\lceil 3\sigma \rceil+1$), and using <code>'reflect'</code> or equivalent boundary conditions.</b></li></ul>
</div>

Complete the $3$ functions in the next $3$ cells. 

In [ ]:
%use sos

# Complete local_mean to calculate the local mean
def local_mean(img, sigma):
    mean = np.zeros(img.shape)
    
    # Calculate local mean
    # YOUR CODE HERE
    return mean

In [ ]:
%use sos

# Complete local_std to calculate the local standard deviation
def local_std(img, mean, sigma):
    s = np.zeros(img.shape)
    
    # Estimate local standard deviation
    # YOUR CODE HERE
    return s

In [ ]:
%use sos

# Complete segment_dic, that normalizes, binarizes and builds an RGB image 
def segment_dic(img, std, t):
    # Declare an image with three channels of the same size as the original
    M, N = img.shape
    output = np.zeros((M, N, 3))
    
    # Normalize std to range [0, 255]
    # YOUR CODE HERE
    
    # Apply thresholding to std
    # YOUR CODE HERE
    
    # Assign thresholded std to blue (2) channel, and original image to red (0) and green (1) channels
    # YOUR CODE HERE
    
    # We make sure to have an int type (this operation floors all elements of the output)
    output = output.astype(int)
    return output

Before visualizing the segmentation, we will run a few sanity checks. In the next cell, we will test that:
 * the output is correctly normalized, 
 * the channels are indeed the binarized standard deviation (blue channel) and original image (red and green channels)
 * the standard deviation is correct (using a $3\times 3$ test image).

In [ ]:
%use sos
# Here we will test on the real image, `hela` the local mean and std
mean = local_mean(hela, 1)
std = local_std(hela, mean, 1)
segmented_hela = segment_dic(hela, std, 10)

# First we test for normalization
if not segmented_hela.min() == 0:
    print(f'WARNING!\nYour minimum value is not correct (it should be 0, instead of {segmented_hela.min()})')
elif not segmented_hela.max() == 255:
    print(f'WARNING!\nYour maximum value is not correct (it should be 255, instead of {segmented_hela.max()})')
else :
    print(f'Well done! Your output seems correctly normalized.')

# Now for binarization
if not len(np.unique(segmented_hela[:, :, 2])) == 2:
    print(f'WARNING!\nYour blue channel should have 2 values, instead of {len(np.unique(segmented_hela[:, :, 2]))}.')

# Now we will check the correctness of the standard deviation with a 3x3 impulse.
impulse = np.array([[0, 0, 0], [0, 255, 0], [0, 0, 0]])

# Apply your function
impulse_mean = local_mean(impulse, 1)
impulse_std = local_std(impulse, impulse_mean, 1)
segmented_impulse = segment_dic(impulse, impulse_std, 10)

# We inspect the blue channel
blue_correct = np.array([[  0, 255,   0],[255, 255, 255],[  0, 255,   0]])
if not np.count_nonzero(segmented_impulse[:,:,2] - blue_correct) == 0:
    print('WARNING!\nYour standard deviation does not seem to be correct!')
else :
    print('Good job! Your implementation passed this initial sanity check.')

Now that we have some sanity checks on the segmentation, we will visualize the local mean and $\sigma$. Make sure to understand both plots! Ask yourself, *what does the local mean represent? What does the local $\sigma$ represent? How do you relate your workflow with the [definition](https://en.wikipedia.org/wiki/Standard_deviation#Uncorrected_sample_standard_deviation) of the standard deviation $\sigma$?* 

Run the next cell to visualize them. 

In [ ]:
%use sos
# Here we will test plot the local mean and std
mean = local_mean(hela, 5)
std = local_std(hela, mean, 5)
segmented_hela = segment_dic(hela, std, 10)

segment_dic_viewer = viewer([hela, mean, std, segmented_hela], title=['Original', 'Mean', 'Standard Dev'], cmap='viridis', widgets=True, hist=True)

(understanding-the-effect-of-the-parameters)=
## 2.B. Understanding the effect of the parameters
[Back to table of contents](#ToC_2_FilteringApplications)

Hopefully, you agree that not a lot of extra information can be extracted from the mean (indeed, it's *just* a Gaussian filtering). But how about the standard deviation? If you implemented the workflow correctly, you should see that it really separates the area of interest from the rest. Let's now look at the role of the two parameters $\sigma$ and $t$. 

:::{note}
So far, we have not discussed the effects of $\sigma$ and of $t$. We have blindly been using $1$ and $10$ respectively. But what constitutes an appropriate parameter? In the next cell, we prepared an interactive viewer with two sliders in the `Extra widgets` menu: 
* The first one will control $\sigma$
* The second one will control `t`. 

Use the button `Segment` to see the effect of the parameters!
:::

In [ ]:
%use sos
# Here we will test plot the local mean and std
mean = local_mean(hela, 1)
std = local_std(hela, mean, 1)
segmented_hela = segment_dic(hela, std, 10)

sigma_slider = widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='$\sigma_1$')
# sigma_slider = widgets.FloatSlider(value=1, min=0.5, max=10.0, step=0.5, description='\u03c3\u2081:')
t_slider = widgets.FloatSlider(value=10, min=1, max=255, step=0.1, description='t')
button = widgets.Button(description = 'Segment')

def segment_callback(img):
    sigma = sigma_slider.value
    t = t_slider.value
    
    mean = local_mean(hela, sigma)
    std = local_std(hela, mean, sigma)
    segmented_hela = segment_dic(hela, std, t)
    return segmented_hela

segment_dic_viewer = viewer(hela, title=['Segmentation'], new_widgets=[sigma_slider, t_slider, button], callbacks=[segment_callback], widgets=True)
button.click()

Congratulations on finishing the second part of the Filtering lab!

:::{attention}
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to [Moodle](https://moodle.epfl.ch/mod/assign/view.php?id=1157357), in a zip file with the other notebook of this lab.

* Keep the name of the notebook as: *2_Filtering_Applications.ipynb*,
* Name the zip file: *Filtering_lab.zip*.
:::